Установка: https://pytorch.org/get-started/locally/

## Сравнение NumPy и PyTorch

Принципы работы в PyTorch немногим отличаются от операций с массивами в NumPy. Рассмотрим следующую функцию ошибки:

$$L=\| y - X\cdot \Theta\|^2_2+\lambda \|\Theta\|_1.$$

Данную функцию можно последовательно реализовать средствами NumPy:

In [2]:
import numpy as np

In [3]:
y = np.random.randn(2, 1)
X = np.random.randn(2, 5)
theta = np.random.randn(5, 1)
lmbda = 0.1

pred = X.dot(theta)
pred_loss = np.sum((y - pred) ** 2)
reg_loss = lmbda * np.sum(np.abs(theta))
loss = pred_loss + reg_loss
print(loss)

6.608734260336121


На PyTorch реализация вычисления функции $L$ выглядит аналогично, однако вместо массивов NumPy используются специальные объекты &ndash; `torch.Tensor`. Создать тензор в PyTorch можно как из простого массива, так и из массива Numpy, при этом указав тип данных, содержащихся в тензоре (по умолчанию float32).

Подробнее о типах данных в PyTorch: https://pytorch.org/docs/stable/tensors.html

In [1]:
import torch

In [4]:
y = torch.Tensor(y)  # из NumPy массива
X = torch.Tensor(X)  # из NumPy массива
theta = torch.Tensor(theta).requires_grad_(True) # для theta понадобится вычислить градиент

pred = X.matmul(theta)
pred_loss = torch.sum((y - pred) ** 2)
reg_loss = lmbda * torch.sum(torch.abs(theta))
loss = pred_loss + reg_loss
print(loss)

tensor(6.6087, grad_fn=<AddBackward0>)


In [5]:
print(torch.Tensor([1, 2, 3]).type())
print(torch.Tensor(np.array([1, 2, 3])).type())
print(torch.LongTensor([1, 2, 3]).type())  # int 64
print(torch.zeros([2, 2, 3], dtype=torch.int64).type())

print(torch.Tensor([[[1, 2, 3], [4, 5, 6]],[[7, 8, 9],[10, 11, 12]]]).size())

torch.FloatTensor
torch.FloatTensor
torch.LongTensor
torch.LongTensor
torch.Size([2, 2, 3])


## Обучение моделей градиентными методами

Обучение моделей в PyTorch производится в цикле. Ниже приведена реализация метода градиентного спуска. Чтобы обучать модели градиентными методами необходимо помнить о том, что на каждой итерации необходимо "обнулять градиенты" (в примере ниже `theta.grad.zero_()`), поскольку по умолчанию PyTorch накапливает градиент, а не вычисляет его заново для текущих значений параметров.

In [6]:
for i in range(10):
    pred = X.matmul(theta)
    pred_loss = torch.sum((y - pred) ** 2)
    reg_loss = lmbda * torch.sum(torch.abs(theta))
    loss = pred_loss + reg_loss

    loss.backward() # вычисляет градиент функции по указанным переменным при их текущих значениях
    theta.data.add_(-0.1*theta.grad.data)  # методы с _ меняют исходный объект
    # add в смысле прибавить, а не добавить элемент к списку
    theta.grad.zero_()  # всегда обнулять градиент (иначе градиенты будут накапливаться)

c:\users\ski6a\appdata\local\programs\python\python37\lib\site-packages\torch\autograd\__init__.py:149: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:115.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


Нижнее подчеркивание у функций перезаписывает значения переменных:

In [7]:
a = torch.Tensor([50, 45, 8])
b = 5

print(a.add(b))
print(a)
print(a.add_(b))
print(a)

tensor([55., 50., 13.])
tensor([50., 45.,  8.])
tensor([55., 50., 13.])
tensor([55., 50., 13.])


## Формирование архитектуры нейронной сети

### Бинарная классификация

В случае бинарной классификации в качестве функции потерь используется, как правило, бинарная кросс-энтропия. Она реализована в PyTorch в виде двух функций: `BCELoss` и `BCEWithLogitsLoss`.

Отличие данных функций состоит в том, что `BCELoss` вычисляет величину ошибки от выхода сети и требует, чтобы в архитектуре явно была указана функция активации на выходном слое. Функция `BCEWithLogitsLoss`, прежде чем вычислять величину потерь, применяет функцию активации (логистическую сигмоиду) к последнему слою, поэтому в архитектуре задавать функцию активации не нужно.

In [8]:
X = torch.FloatTensor(np.random.randn(5, 3))
y = torch.FloatTensor(np.random.randint(0, 2, 5)) # float32

#### Архитектура с `BCELoss`

In [10]:
network = torch.nn.Sequential(
    torch.nn.Linear(3, 10),  # соидиняет слои;
    # torch.nn.Linear(a, b) число нейронов на  (входе; выходе) ^
    torch.nn.Sigmoid(),  # функция активации
    torch.nn.Linear(10, 20),  # след. слой будет из 20 нейронов
    torch.nn.Sigmoid(),  # функция активации
    torch.nn.Linear(20, 1),  # последний слой из одного нейрона
    torch.nn.Sigmoid() # явно задана функция активации
)

optimizer = torch.optim.SGD(network.parameters(), lr=0.01, weight_decay=0.05)
# weight_decay - регуляризация lambda*sum(theta**2) | lambda = weight_decay
criterion = torch.nn.BCELoss()  # функция потерь

for i in range(100):
    optimizer.zero_grad()
    pred = network(X) # выдает вероятности
    loss = criterion(pred, y.unsqueeze(1)) # размерность pred (5,1); размерность y - (5) 
    # преобразовывает y к размерности 5
    loss.backward()  # обратное распространение ошибки (градиенты на всех слоях)
    optimizer.step()  # шаг градиентного метода (с учетом градиентов на весах)

In [12]:
print(pred) # возвращает вероятности отношения объекта к классу 1

tensor([[0.4241],
        [0.4280],
        [0.4214],
        [0.4216],
        [0.4218]], grad_fn=<SigmoidBackward>)


#### Пример с `BCEWithLogitsLoss`

In [13]:
network = torch.nn.Sequential(
    torch.nn.Linear(3, 10), 
    torch.nn.Sigmoid(), 
    torch.nn.Linear(10, 20), 
    torch.nn.Sigmoid(),
    torch.nn.Linear(20, 1) # последний слой - линейный
)

# та же самая архиетктура, но последний слой линейных
# в вероятности это будет преобразовывать сам BCEWithLogitsLoss()
# сам брать сигмоиду и т.д.
# и во всех остальных функциях потреь также

optimizer = torch.optim.SGD(network.parameters(), lr=0.01, weight_decay=0.05)
criterion = torch.nn.BCEWithLogitsLoss()

for i in range(100):
    optimizer.zero_grad()
    pred = network(X)
    loss = criterion(pred, y.unsqueeze(1)) # размерности для BCEWithLogitsLoss должны совпадать
    loss.backward()
    optimizer.step()

In [14]:
print(pred) # возвращает выход последнего линейного слоя
print(torch.sigmoid(pred)) # преобразование в вероятности

tensor([[-0.2700],
        [-0.2515],
        [-0.2353],
        [-0.2629],
        [-0.2587]], grad_fn=<AddmmBackward>)
tensor([[0.4329],
        [0.4374],
        [0.4415],
        [0.4346],
        [0.4357]], grad_fn=<SigmoidBackward>)


### Многоклассовая классификация

Бинарная классификация &ndash; частный случай многоклассовой классификации. Для многоклассовой классификации используется кросс-энтропия. Функция потерь `CrossEntropyLoss` также не требует указания функции активации последним слоем в архитектуре сети.

In [26]:
X = torch.FloatTensor(np.random.randn(5, 3))
y = torch.LongTensor(np.random.randint(0, 2, 5)) # CrossEntropyLoss требует формата int64

In [27]:
y

tensor([0, 0, 0, 0, 1])

In [28]:
network = torch.nn.Sequential(
    torch.nn.Linear(3, 10), 
    torch.nn.Sigmoid(), 
    torch.nn.Linear(10, 20), 
    torch.nn.Sigmoid(),
    torch.nn.Linear(20, 2) # на выходном слое 2 нейрона
)

optimizer = torch.optim.SGD(network.parameters(), lr=0.01, weight_decay=0.05)
criterion = torch.nn.CrossEntropyLoss() # применяет softmax к последнему слою и вычисляет ошибку

for i in range(100):
    optimizer.zero_grad()
    pred = network(X)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()

In [29]:
print(torch.softmax(pred, 1)) # преобразование в вероятности
# функция активации применяется вручную

tensor([[0.7742, 0.2258],
        [0.7758, 0.2242],
        [0.7742, 0.2258],
        [0.7742, 0.2258],
        [0.7745, 0.2255]], grad_fn=<SoftmaxBackward>)


#### Случай нескольких классов

In [30]:
X = torch.FloatTensor(np.random.randn(5, 3))
y = torch.LongTensor(np.random.randint(0, 3, 5))

In [31]:
print(y)

tensor([1, 0, 2, 1, 2])


In [32]:
network = torch.nn.Sequential(
    torch.nn.Linear(3, 10), 
    torch.nn.Sigmoid(), 
    torch.nn.Linear(10, 20), 
    torch.nn.Sigmoid(),
    torch.nn.Linear(20, 3) # 3 нейрона, соответствующие 3 классам
)

optimizer = torch.optim.SGD(network.parameters(), lr=0.01, weight_decay=0.05)
criterion = torch.nn.CrossEntropyLoss()
# сама берет soft max и считает значение кросс-энтропии, поэтому выше
# последний слой линейный

for i in range(100):
    optimizer.zero_grad()
    pred = network(X)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()

In [33]:
print(torch.softmax(pred, 1))  # функция активации применяется вручную

tensor([[0.2152, 0.3844, 0.4004],
        [0.2223, 0.3777, 0.4001],
        [0.2210, 0.3768, 0.4022],
        [0.2237, 0.3777, 0.3987],
        [0.2166, 0.3806, 0.4029]], grad_fn=<SoftmaxBackward>)


Поэтому для бинарной классификации стоит выбирать функцию потерь также, чтобы была единость. То есть чтобы последний слой в архитектуре сети был линейным.

## Оформление архитектуры в виде класса

In [34]:
class NeuralNetwork(torch.nn.Module):
    
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.layer_1 = torch.nn.Linear(3, 20)
        self.layer_2 = torch.nn.Linear(20, 10)
        self.layer_out = torch.nn.Linear(10, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
        # если мы пишем через класс, то нам достаточно
        # один раз прописать сигмоиду
        
    def forward(self, inputs):
        output_1 = self.sigmoid(self.layer_1(inputs))
        output_2 = self.sigmoid(self.layer_2(output_1))
        output = self.layer_out(output_2)
        
        return output

In [35]:
model = NeuralNetwork()
print(model)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

NeuralNetwork(
  (layer_1): Linear(in_features=3, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=10, bias=True)
  (layer_out): Linear(in_features=10, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [36]:
X = torch.FloatTensor(np.random.randn(5,3))
y = torch.FloatTensor(np.random.randint(0,2,5))

for i in range(10):
    optimizer.zero_grad()
    pred = model(X)
    loss = criterion(pred, y.unsqueeze(1))
    print('iter: ', i+1,' loss: ', loss.item())
    loss.backward()
    optimizer.step()

iter:  1  loss:  0.7446807622909546
iter:  2  loss:  0.7316052913665771
iter:  3  loss:  0.7209020853042603
iter:  4  loss:  0.7121487259864807
iter:  5  loss:  0.7049938440322876
iter:  6  loss:  0.6991468071937561
iter:  7  loss:  0.6943684220314026
iter:  8  loss:  0.6904624104499817
iter:  9  loss:  0.6872681975364685
iter:  10  loss:  0.6846548914909363


<div class="alert alert-info">

<h3> Упражнение</h3>
<p></p>
Реализовать и обучить модели из предыдущих заданий (binary_classification.csv, multiclass_classification.csv, светофор)
 <p></p>
</div>

In [37]:
import pandas as pd

In [46]:
multy_data = pd.read_csv('data/multiclass_classification.csv')
binary_data = pd.read_csv('data/task2.csv')

**Модель для данных с двумя классами**

In [91]:
class BinaryNeuralNetwork(torch.nn.Module):
    
    def __init__(self):
        super(BinaryNeuralNetwork, self).__init__()
        
        self.layer_1 = torch.nn.Linear(2, 3)
        self.layer_out = torch.nn.Linear(3, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, inputs):
        output_1 = self.sigmoid(self.layer_1(inputs))        
        output = self.layer_out(output_1)
        
        return output

In [92]:
model = BinaryNeuralNetwork()
print(model)

criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

BinaryNeuralNetwork(
  (layer_1): Linear(in_features=2, out_features=3, bias=True)
  (layer_out): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [93]:
Xb, yb = binary_data.values[:, :-1], binary_data.values[:, -1]
Xb_train, Xb_test, yb_train, yb_test = Xb[:-5, :], Xb[-5:, :], yb[:-5], yb[-5:]

In [98]:
X = torch.FloatTensor(Xb_train)
y = torch.FloatTensor(yb_train)

for i in range(1000):
    optimizer.zero_grad()
    pred = model(X)
    loss = criterion(pred, y.unsqueeze(1))
    print('iter: ', i+1,' loss: ', loss.item())
    loss.backward()
    optimizer.step()

iter:  1  loss:  0.2988814413547516
iter:  2  loss:  0.2887113690376282
iter:  3  loss:  0.27870890498161316
iter:  4  loss:  0.2688693106174469
iter:  5  loss:  0.2591882050037384
iter:  6  loss:  0.2496614158153534
iter:  7  loss:  0.2402849793434143
iter:  8  loss:  0.23105524480342865
iter:  9  loss:  0.2219686657190323
iter:  10  loss:  0.213021919131279
iter:  11  loss:  0.20421180129051208
iter:  12  loss:  0.19553519785404205
iter:  13  loss:  0.18698902428150177
iter:  14  loss:  0.17857033014297485
iter:  15  loss:  0.17027613520622253
iter:  16  loss:  0.16210362315177917
iter:  17  loss:  0.15404990315437317
iter:  18  loss:  0.14611214399337769
iter:  19  loss:  0.13828745484352112
iter:  20  loss:  0.13057316839694977
iter:  21  loss:  0.1229664757847786
iter:  22  loss:  0.11546465754508972
iter:  23  loss:  0.10806506127119064
iter:  24  loss:  0.10076489299535751
iter:  25  loss:  0.09356166422367096
iter:  26  loss:  0.08645272254943848
iter:  27  loss:  0.07943546026

iter:  217  loss:  -0.7407318353652954
iter:  218  loss:  -0.7441412210464478
iter:  219  loss:  -0.7475472092628479
iter:  220  loss:  -0.7509498000144958
iter:  221  loss:  -0.7543491125106812
iter:  222  loss:  -0.757745087146759
iter:  223  loss:  -0.761137843132019
iter:  224  loss:  -0.7645274996757507
iter:  225  loss:  -0.7679139375686646
iter:  226  loss:  -0.7712970972061157
iter:  227  loss:  -0.7746772766113281
iter:  228  loss:  -0.7780542373657227
iter:  229  loss:  -0.7814283967018127
iter:  230  loss:  -0.7847993969917297
iter:  231  loss:  -0.788167417049408
iter:  232  loss:  -0.791532576084137
iter:  233  loss:  -0.794894814491272
iter:  234  loss:  -0.7982542514801025
iter:  235  loss:  -0.8016107082366943
iter:  236  loss:  -0.8049644827842712
iter:  237  loss:  -0.8083153963088989
iter:  238  loss:  -0.8116637468338013
iter:  239  loss:  -0.8150091767311096
iter:  240  loss:  -0.8183520436286926
iter:  241  loss:  -0.821692168712616
iter:  242  loss:  -0.825029790

iter:  512  loss:  -1.6898815631866455
iter:  513  loss:  -1.6930835247039795
iter:  514  loss:  -1.6962867975234985
iter:  515  loss:  -1.699489951133728
iter:  516  loss:  -1.702694296836853
iter:  517  loss:  -1.7058990001678467
iter:  518  loss:  -1.709104299545288
iter:  519  loss:  -1.712310552597046
iter:  520  loss:  -1.715517282485962
iter:  521  loss:  -1.7187243700027466
iter:  522  loss:  -1.7219325304031372
iter:  523  loss:  -1.725140929222107
iter:  524  loss:  -1.728350281715393
iter:  525  loss:  -1.731560468673706
iter:  526  loss:  -1.7347713708877563
iter:  527  loss:  -1.7379823923110962
iter:  528  loss:  -1.7411949634552002
iter:  529  loss:  -1.7444075345993042
iter:  530  loss:  -1.7476211786270142
iter:  531  loss:  -1.7508351802825928
iter:  532  loss:  -1.754050374031067
iter:  533  loss:  -1.7572659254074097
iter:  534  loss:  -1.7604824304580688
iter:  535  loss:  -1.7636996507644653
iter:  536  loss:  -1.7669175863265991
iter:  537  loss:  -1.770135760307

iter:  812  loss:  -2.702906847000122
iter:  813  loss:  -2.7065556049346924
iter:  814  loss:  -2.7102081775665283
iter:  815  loss:  -2.713862419128418
iter:  816  loss:  -2.717519521713257
iter:  817  loss:  -2.721179723739624
iter:  818  loss:  -2.7248423099517822
iter:  819  loss:  -2.7285079956054688
iter:  820  loss:  -2.732175827026367
iter:  821  loss:  -2.735846757888794
iter:  822  loss:  -2.739520311355591
iter:  823  loss:  -2.7431960105895996
iter:  824  loss:  -2.746875286102295
iter:  825  loss:  -2.7505569458007812
iter:  826  loss:  -2.754241704940796
iter:  827  loss:  -2.7579288482666016
iter:  828  loss:  -2.7616188526153564
iter:  829  loss:  -2.7653112411499023
iter:  830  loss:  -2.7690064907073975
iter:  831  loss:  -2.772704601287842
iter:  832  loss:  -2.7764053344726562
iter:  833  loss:  -2.78010892868042
iter:  834  loss:  -2.7838151454925537
iter:  835  loss:  -2.7875239849090576
iter:  836  loss:  -2.79123592376709
iter:  837  loss:  -2.794950485229492
i

In [ ]:
print(torch.softmax(pred, 1))

**Модель для данных с многими классами**

In [102]:
Xm, ym = multy_data.values[:, :-1], multy_data.values[:, -1]
ym = np.where(ym == 3, 0, ym)
Xm_train, Xm_test, ym_train, ym_test = Xm[:-5, :], Xm[-5:, :], ym[:-5], ym[-5:]

In [103]:
np.unique(ym)

array([0., 1., 2.])

In [104]:
X = torch.FloatTensor(Xm_train)
y = torch.FloatTensor(ym_train)

In [105]:
class MultyNeuralNetwork(torch.nn.Module):
    
    def __init__(self):
        super(MultyNeuralNetwork, self).__init__()
        
        self.layer_1 = torch.nn.Linear(2, 3)
        self.layer_2 = torch.nn.Linear(3, 4)
        self.layer_out = torch.nn.Linear(4, 3)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, inputs):
        output_1 = self.sigmoid(self.layer_1(inputs))
        output_2 = self.sigmoid(self.layer_1(output_1))
        output = self.layer_out(output_2)
        
        return output

In [106]:
model = MultyNeuralNetwork()
print(model)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=0.01, weight_decay=0.05)

MultyNeuralNetwork(
  (layer_1): Linear(in_features=2, out_features=3, bias=True)
  (layer_2): Linear(in_features=3, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=3, bias=True)
  (sigmoid): Sigmoid()
)


In [108]:
for i in range(100):
    optimizer.zero_grad()
    pred = network(X)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()
loss

RuntimeError: mat1 and mat2 shapes cannot be multiplied (995x2 and 3x10)